In [25]:
import pandas as pd
import nfl_data_py as nfl
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from pulp import *
import gurobipy as gp
from itertools import cycle
from gurobipy import *

In [26]:
# laden der Daten
player_data = pd.read_csv('/Users/mariu/VS Code Repositories/data-driven-fantasy-manager/player_data.csv')
players_rank = pd.read_csv('/Users/mariu/VS Code Repositories/data-driven-fantasy-manager/rank.csv')

In [27]:
# Erstellen eines Spieplans
def create_round_robin(players):
    n = len(players)
    schedule = []

    for round in range(n-1):
        for i in range(n//2):
            home = players[i]
            away = players[n - i - 1 - round]
            if i % 2 == round % 2:
                home, away = away, home
            schedule.append((round + 1, home, away))

    return schedule

def add_extra_games(schedule, players):
    n = len(players)
    rounds = len(schedule) // (n // 2)
    
    for round in range(rounds, rounds + (n - 1)//2):
        for i in range(n//2):
            home = players[i]
            away = players[n - i - 1]
            if i % 2 == round % 2:
                home, away = away, home
            schedule.append((round + 1, home, away))
    
    return schedule

managers = ['Manager_' + str(i) for i in range(1, 11)]

round_robin_schedule = create_round_robin(managers)
full_schedule = add_extra_games(round_robin_schedule, managers)

manager_schedule = pd.DataFrame(full_schedule, columns=['week', 'Manager', 'Opponent'])
manager_schedule.sort_values(by=['week', 'Manager'], inplace=True)

In [28]:

player_data.set_index(['player_id', 'week'], inplace=True)

# Erstellen einer Matrix der erwarteten Punkte
expected_points = player_data['predicted_fantasy_points'].unstack()

# NaN-Werte ersetzen (falls ein Spieler in einer bestimmten Woche nicht spielt)
expected_points.fillna(0, inplace=True)


player_data = player_data.reset_index()
expected_points_long = expected_points.reset_index().melt(id_vars='player_id', var_name='week', value_name='points')
expected_points_long['week'] = expected_points_long['week'].astype(int)  # Stellen Sie sicher, dass 'week' ein integer ist
expected_points_dict = {(row['player_id'], row['week']): row['points'] for index, row in expected_points_long.iterrows()}


In [29]:
# Erstellen eines
teams = ['Manager_' + str(i+1) for i in range(10)]

# Die Anfangsreihenfolge für den Draft kann zufällig oder auf eine andere Weise festgelegt werden
initial_draft_order = teams

# Erstelle die Snake-Draft-Order für alle Runden
draft_order = []
for r in range(16): # Angenommen, wir haben 16 Runden im Draft
    if r % 2 == 0:
        draft_order.append(initial_draft_order)
    else:
        draft_order.append(initial_draft_order[::-1]) # Die Reihenfolge umkehren


In [30]:


# Erstelle einen round-robin Spielplan aus Sicht von Manager_1
schedule = {}
rotating_teams = teams[1:] # 'Manager_1' ist fixiert, die anderen Teams rotieren

for week in range(1, 14): # Angenommen, wir haben eine 14-wöchige Saison
    rotating_teams = rotating_teams[-1:] + rotating_teams[:-1] # rotiere die Teams
    matchups = list(zip(['Manager_1'] + rotating_teams[:len(rotating_teams)//2], rotating_teams[len(rotating_teams)//2:]))
    
    for team1, team2 in matchups:
        if team1 not in schedule:
            schedule[team1] = {}
        if team2 not in schedule:
            schedule[team2] = {}
        schedule[team1][week] = team2
        schedule[team2][week] = team1


In [31]:
players_rank = players_rank.sort_values(by='rank')
players_rank = players_rank[['player_id', 'Player', 'POS', 'rank']]


In [32]:
# Ausführen des Modells wenn Ranking Variante 1 oder 2 
# Liste der Spieler und Teams erstellen
players = player_data['player_id'].unique().tolist()
teams = manager_schedule['Manager'].unique().tolist()
weeks = list(range(1, 14)) 
position = dict(zip(player_data['player_id'], player_data['position_group']))

# Angenommen, jedes Team wählt 16 Spieler
num_players_per_team = 16
rounds = list(range(1, num_players_per_team+1))

# Erstelle ein dictionary, das den Rang für jeden Spieler über die gesamte Saison speichert
season_ranking = {row['player_id']: row['average_rank'] for index, row in players_rank.iterrows()}

# Initialisieren Sie das Modell
m = Model("Fantasy_Football_Optimization")

# x[i][j][t] ist 1, wenn Spieler i in Woche j von Team t ausgewählt und aufgestellt wird
x = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="chosen")

# y[i][t] ist 1, wenn Spieler i von Team t für die gesamte Saison ausgewählt wird
y = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# z[t][r] ist 1, wenn Team t in Runde r einen Spieler auswählt
z = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][j] ist 1, wenn Team t in Woche j gewinnt
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Zielfunktion: Maximiere die Anzahl der Siege von Manager_1
m.setObjective(quicksum(w['Manager_1', j] for j in weeks), GRB.MAXIMIZE)

# Spielplan-Nebenbedingungen
for j in weeks:
    opponent = schedule['Manager_1'][j]
    m.addConstr(quicksum(x[i, j, 'Manager_1'] * expected_points_dict[(i, j)] for i in players) - 
                quicksum(x[i, j, opponent] * expected_points_dict[(i, j)] for i in players) >= 
                w['Manager_1', j] - (1 - w[opponent, j]))

for t in teams:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for i in players) == 9)
# Nebenbedingungen hinzufügen
# Einschränkungen für die Anzahl der Spieler pro Position
for t in teams:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'QB') == 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] in ['WR', 'RB']) + 
                    quicksum(x[i, j, t] for i in players if position[i] == 'TE') >= 3)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'RB') >= 2)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'TE') >= 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'K') == 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'DST') == 1)

for i in players:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for t in teams) <= 1)

for i in players:
    for t in teams:
        m.addConstr(y[i, t] >= quicksum(x[i, j, t] for j in weeks) / len(weeks))

for i in players:
    m.addConstr(quicksum(y[i, t] for t in teams) <= 1)

for t in teams:
    if t != 'Manager_1':
        m.addConstr(quicksum(y[i, t] * season_ranking[i] for i in players if i in season_ranking and (i, t) in y) <= len(players))

for t in teams:
    m.addConstr(quicksum(y[i, t] for i in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(z[t, r] == 1)

# Problem lösen
m.optimize()

# Ausgabe der Ergebnisse
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

print('Total wins: %g' % m.objVal)

# Extrahieren der ausgewählten Spieler für jedes Team und jede Woche
selected_players = [(i, j, t) for i in players for j in weeks for t in teams if x[i, j, t].x == 1]

# Umwandlung der Liste in ein DataFrame für eine bessere Darstellung
team_df = pd.DataFrame(selected_players, columns=['Player ID', 'Week', 'Team'])

team_df


KeyError: 'average_rank'

In [33]:
# Ausführen des Modells, wenn Ranking Variante 3

# Liste der Spieler und Teams erstellen
players = player_data['player_id'].unique().tolist()
teams = manager_schedule['Manager'].unique().tolist()
weeks = list(range(1, 14)) 
position = dict(zip(player_data['player_id'], player_data['position_group']))

# Angenommen, jedes Team wählt 15 Spieler
num_players_per_team = 16
rounds = list(range(1, num_players_per_team+1))

# Erstelle ein dictionary, das den Rang für jeden Spieler über die gesamte Saison speichert
season_ranking = {row['player_id']: row['rank'] for index, row in players_rank.iterrows()}

# Initialisieren Sie das Modell
m = Model("Fantasy_Football_Optimization")

# x[i][j][t] ist 1, wenn Spieler i in Woche j von Team t ausgewählt und aufgestellt wird
x = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="chosen")

# y[i][t] ist 1, wenn Spieler i von Team t für die gesamte Saison ausgewählt wird
y = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# z[t][r] ist 1, wenn Team t in Runde r einen Spieler auswählt
z = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][j] ist 1, wenn Team t in Woche j gewinnt
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Zielfunktion: Maximiere die Anzahl der Siege von Manager_1
m.setObjective(quicksum(w['Manager_1', j] for j in weeks), GRB.MAXIMIZE)

# Spielplan-Nebenbedingungen
for j in weeks:
    opponent = schedule['Manager_1'][j]
    m.addConstr(quicksum(x[i, j, 'Manager_1'] * expected_points_dict[(i, j)] for i in players) - 
                quicksum(x[i, j, opponent] * expected_points_dict[(i, j)] for i in players) >= 
                w['Manager_1', j] - (1 - w[opponent, j]))

for t in teams:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for i in players) == 9)
# Nebenbedingungen hinzufügen
# Ändern Sie die Einschränkungen für die Anzahl der Spieler pro Position
for t in teams:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'QB') == 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] in ['WR', 'RB']) + 
                    quicksum(x[i, j, t] for i in players if position[i] == 'TE') >= 3)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'RB') >= 2)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'TE') >= 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'K') == 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'DST') == 1)

for i in players:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for t in teams) <= 1)

for i in players:
    for t in teams:
        m.addConstr(y[i, t] >= quicksum(x[i, j, t] for j in weeks) / len(weeks))

for i in players:
    m.addConstr(quicksum(y[i, t] for t in teams) <= 1)

for t in teams:
    if t != 'Manager_1':
        m.addConstr(quicksum(y[i, t] * season_ranking[i] for i in players if i in season_ranking and (i, t) in y) <= len(players))

for t in teams:
    m.addConstr(quicksum(y[i, t] for i in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(z[t, r] == 1)

# Problem lösen
m.optimize()

# Ausgabe der Ergebnisse
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

print('Total wins: %g' % m.objVal)

# Extrahieren der ausgewählten Spieler für jedes Team und jede Woche
selected_players = [(i, j, t) for i in players for j in weeks for t in teams if x[i, j, t].x == 1]

# Umwandlung der Liste in ein DataFrame für eine bessere Darstellung
team_df = pd.DataFrame(selected_players, columns=['Player ID', 'Week', 'Team'])

team_df


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 11878 rows, 63150 columns and 282922 nonzeros
Model fingerprint: 0xd81ed74b
Variable types: 0 continuous, 63150 integer (63150 binary)
Coefficient statistics:
  Matrix range     [8e-02, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 612 rows and 2041 columns
Presolve time: 0.83s
Presolved: 11266 rows, 61109 columns, 218716 nonzeros
Variable types: 0 continuous, 61109 integer (61109 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.09s

Solved with primal simplex

Root relaxation: objective 1.300000e+01, 7413 iterations, 0.39 seconds (0.43 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | 

,Player ID,Week,Team
0,00-0019596,1,Manager_1
1,00-0019596,2,Manager_1
2,00-0019596,3,Manager_1
3,00-0019596,4,Manager_1
4,00-0019596,5,Manager_1
...,...,...,...
1165,00-000011,9,Manager_5
1166,00-000011,10,Manager_5
1167,00-000011,11,Manager_5
1168,00-000011,12,Manager_5


In [ ]:
# Anzeigen der Punkte, die in der jeweiligen Woche erzielt werden von den verschiedenen Managern
results_df = pd.merge(team_df, player_data[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

results_df = results_df.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

results_df = results_df.rename(columns={'fantasy_points_ppr': 'Total Points'})

print(results_df[results_df['Team'] == 'Manager_1'])

print(results_df[results_df['Team'] != 'Manager_1'])


In [ ]:
# Anzeigen der Spiele von Manager_1 mit Ergebnissen
schedule_df = pd.concat({k: pd.Series(v) for k, v in schedule.items()}).reset_index()

schedule_df.columns = ['Team', 'Week', 'Opponent']

results_with_opponents = pd.merge(results_df, schedule_df, how='left', on=['Team', 'Week'])

results_with_opponents['Opponent Points'] = results_with_opponents.apply(
    lambda row: results_df[
        (results_df['Team'] == row['Opponent']) &
        (results_df['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in results_df['Team'].values else 0,
    axis=1
)

results_with_opponents['Won'] = results_with_opponents['Total Points'] > results_with_opponents['Opponent Points']

manager_1_results = results_with_opponents[results_with_opponents['Team'] == 'Manager_1']

print(manager_1_results[['Week', 'Total Points', 'Opponent', 'Opponent Points', 'Won']])


In [34]:
# Ausgabe der ausgewählten Spieler für jedes Team
for t in teams:
    print(f'Team {t} hat folgende Spieler ausgewählt:')
    for i in players:
        if y[i, t].x > 0:
            print(f'Player ID: {i}')


Team Manager_10 hat folgende Spieler ausgewählt:
Player ID: 00-0022924
Player ID: 00-0026189
Player ID: 00-0027150
Player ID: 00-0027685
Player ID: 00-0032144
Player ID: 00-0032661
Player ID: 00-0033594
Player ID: 00-0033873
Player ID: 00-0033897
Player ID: 00-0033906
Player ID: 00-0034766
Player ID: 00-0032569
Player ID: 00-000002
Team Manager_2 hat folgende Spieler ausgewählt:
Player ID: 00-0025394
Player ID: 00-0026158
Player ID: 00-0027944
Player ID: 00-0029137
Player ID: 00-0031687
Player ID: 00-0032055
Player ID: 00-0032688
Player ID: 00-0032764
Player ID: 00-0032765
Player ID: 00-0033881
Player ID: 00-0033932
Player ID: 00-0034827
Player ID: 00-0029822
Player ID: 00-000005
Team Manager_4 hat folgende Spieler ausgewählt:
Player ID: 00-0024243
Player ID: 00-0027939
Player ID: 00-0029632
Player ID: 00-0030061
Player ID: 00-0030279
Player ID: 00-0033681
Player ID: 00-0034109
Player ID: 00-0034301
Player ID: 00-0034772
Player ID: 00-0026858
Player ID: 00-000001
Team Manager_6 hat fol

In [35]:
# Liste für die ausgewählten Spieler-Daten
selected_players_data = []

for t in teams:
    for i in players:
        if y[i, t].x > 0:
            selected_players_data.append({
                'Team': t,
                'Player ID': i,
                'Player Name': player_data.loc[player_data['player_id'] == i, 'player_display_name'].iloc[0],
                'Position': player_data.loc[player_data['player_id'] == i, 'position_group'].iloc[0]
            })

selected_players_df = pd.DataFrame(selected_players_data)

selected_players_df


,Team,Player ID,Player Name,Position
0,Manager_10,00-0022924,Ben Roethlisberger,QB
1,Manager_10,00-0026189,DeSean Jackson,WR
2,Manager_10,00-0027150,Julian Edelman,WR
3,Manager_10,00-0027685,Emmanuel Sanders,WR
4,Manager_10,00-0032144,Melvin Gordon,RB
...,...,...,...,...
115,Manager_9,00-0033118,Kenyan Drake,RB
116,Manager_9,00-0033127,Will Fuller,WR
117,Manager_9,00-0034837,Calvin Ridley,WR
118,Manager_9,00-0033303,Harrison Butker,K


In [36]:
# Berechnen Sie die Punkte für jedes Spiel
for j in weeks:
    print(f'Week {j}:')
    for t in teams:
        points = sum(x[i, j, t].x * expected_points_dict[(i, j)] for i in players)
        print(f'Team {t} scored {points} points.')


Week 1:
Team Manager_10 scored 96.901038 points.
Team Manager_2 scored 88.9048189 points.
Team Manager_4 scored 77.49084280000001 points.
Team Manager_6 scored 92.9123016 points.
Team Manager_8 scored 68.13256129999999 points.
Team Manager_1 scored 82.4533631 points.
Team Manager_3 scored 74.93793519999998 points.
Team Manager_5 scored 71.2343725 points.
Team Manager_7 scored 103.57464670000002 points.
Team Manager_9 scored 105.48677260000001 points.
Week 2:
Team Manager_10 scored 81.209644 points.
Team Manager_2 scored 86.784051 points.
Team Manager_4 scored 66.0581576 points.
Team Manager_6 scored 86.36071000000001 points.
Team Manager_8 scored 85.62790469999999 points.
Team Manager_1 scored 72.28160520000002 points.
Team Manager_3 scored 68.5814197 points.
Team Manager_5 scored 82.4241025 points.
Team Manager_7 scored 102.714523 points.
Team Manager_9 scored 103.8670025 points.
Week 3:
Team Manager_10 scored 61.9604018 points.
Team Manager_2 scored 75.96344 points.
Team Manager_4 sc